In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("ALS").setMaster("local[4]")
spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

In [2]:
global Path    
if sc.master[0:5]=="local" :
   Path="/mnt/data1/workspace/data_analysis_mining/Python_Spark/pythonsparkexample/PythonProject"
else:   
   Path="hdfs://master:9000/user/hduser/"

In [3]:
from pyspark.ml.linalg import Vectors, SparseMatrix

In [5]:
SparseMatrix? 
# 列压缩

Init signature:
SparseMatrix(
    numRows,
    numCols,
    colPtrs,
    rowIndices,
    values,
    isTransposed=False,
)
Docstring:      Sparse Matrix stored in CSC format.
File:           /usr/local/spark/python/pyspark/ml/linalg/__init__.py
Type:           type
Subclasses:     


In [4]:
from scipy.sparse import csc_matrix
import numpy as np

In [5]:
csc_matrix?

Init signature: csc_matrix(arg1, shape=None, dtype=None, copy=False)
Docstring:     
Compressed Sparse Column matrix

This can be instantiated in several ways:

    csc_matrix(D)
        with a dense matrix or rank-2 ndarray D

    csc_matrix(S)
        with another sparse matrix S (equivalent to S.tocsc())

    csc_matrix((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csc_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csc_matrix((data, indices, indptr), [shape=(M, N)])
        is the standard CSC representation where the row indices for
        column i are stored in ``indices[indptr[i]:indptr[i+1]]``
        and their corresponding values are stored in
        ``data[indptr[i]:indptr[i+1]]``.  If the shape parameter is
        not supplied, the matrix dimensions are infe

矩阵中第j列非零元素的行号为indices[indptr[i]:indptr[i+1]]，相应的值为data[indptr[i]:indptr[i+1]]

In [6]:
row = np.array([0, 2, 2, 0, 1, 2])
col = np.array([0, 0, 1, 2, 2, 2])
data = np.array([1, 2, 3, 4, 5, 6])

matrix = csc_matrix((data, (row, col)), shape=(3, 3))
matrix.toarray()

array([[1, 0, 4],
       [0, 0, 5],
       [2, 3, 6]], dtype=int64)

In [7]:
matrix.indices

array([0, 2, 2, 0, 1, 2], dtype=int32)

In [8]:
matrix.indptr

array([0, 2, 3, 6], dtype=int32)

In [9]:
matrix.data

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [10]:
ss_csc =  SparseMatrix(
    3, 
    3, 
    colPtrs = matrix.indptr,
    rowIndices=matrix.indices,
    values=matrix.data
    )

In [11]:
ss_csc.toArray()

array([[1., 0., 4.],
       [0., 0., 5.],
       [2., 3., 6.]])

In [12]:
ss_csc.toDense()

DenseMatrix(3, 3, [1.0, 0.0, 2.0, 0.0, 0.0, 3.0, 4.0, 5.0, 6.0], False)

In [44]:
data = spark.read.csv(
    os.path.join(Path, 'data/u.data'),sep='\t')

In [45]:
data.show(3)

+---+---+---+---------+
|_c0|_c1|_c2|      _c3|
+---+---+---+---------+
|196|242|  3|881250949|
|186|302|  3|891717742|
| 22|377|  1|878887116|
+---+---+---+---------+
only showing top 3 rows



In [46]:
data = data.withColumnRenamed('_c0', 'user').withColumnRenamed('_c1', 'item').withColumnRenamed('_c2', 'rating').select('user', 'item', 'rating')
data.show(3)

+----+----+------+
|user|item|rating|
+----+----+------+
| 196| 242|     3|
| 186| 302|     3|
|  22| 377|     1|
+----+----+------+
only showing top 3 rows



In [49]:
data.head().rating

'3'

In [76]:
float('3')

3.0

In [13]:
raw_user_data = sc.textFile(os.path.join(Path, 'data/u.data'))
ratings_rdd = raw_user_data \
        .map(lambda line: line.split('\t')[:3]) \
        .map(
            lambda x: (
                    int(x[0]), int(x[1]), float(x[2])
                )
        )

In [14]:
from pyspark.sql.types import *

schema = StructType(fields=[
    StructField('user', IntegerType(), True),
    StructField('item', IntegerType(), True),
    StructField('rating', FloatType(), True),
])
data = spark.createDataFrame(ratings_rdd, schema)

In [15]:
data.printSchema()

root
 |-- user: integer (nullable = true)
 |-- item: integer (nullable = true)
 |-- rating: float (nullable = true)



In [16]:
data.show(3)

+----+----+------+
|user|item|rating|
+----+----+------+
| 196| 242|   3.0|
| 186| 302|   3.0|
|  22| 377|   1.0|
+----+----+------+
only showing top 3 rows



In [17]:
from pyspark.ml.recommendation import ALS

als = ALS(rank=10, maxIter=5, seed=0)

In [18]:
train_df, test_df = data.randomSplit([0.7, 0.3])

In [19]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

In [ ]:
als = ALS(rank=10, maxIter=5, seed=0)

In [20]:
train_df.count()

69944

In [21]:
# rank,迭代次数, 正则化, 置信参数
grid_search = ParamGridBuilder()\
            .addGrid(als.rank, [10, 20, 30]) \
            .addGrid(als.maxIter, [5, 10, 15]) \
            .addGrid(als.regParam, [0.1, 0.01, 1]) \
            .addGrid(als.alpha, [0.5, 1., 10.]) \
            .build()

In [102]:
RegressionEvaluator?

Init signature:
RegressionEvaluator(
    predictionCol='prediction',
    labelCol='label',
    metricName='rmse',
)
Docstring:     
.. note:: Experimental

Evaluator for Regression, which expects two input
columns: prediction and label.

>>> scoreAndLabels = [(-28.98343821, -27.0), (20.21491975, 21.5),
...   (-25.98418959, -22.0), (30.69731842, 33.0), (74.69283752, 71.0)]
>>> dataset = spark.createDataFrame(scoreAndLabels, ["raw", "label"])
...
>>> evaluator = RegressionEvaluator(predictionCol="raw")
>>> evaluator.evaluate(dataset)
2.842...
>>> evaluator.evaluate(dataset, {evaluator.metricName: "r2"})
0.993...
>>> evaluator.evaluate(dataset, {evaluator.metricName: "mae"})
2.649...
>>> re_path = temp_path + "/re"
>>> evaluator.save(re_path)
>>> evaluator2 = RegressionEvaluator.load(re_path)
>>> str(evaluator2.getPredictionCol())
'raw'

.. versionadded:: 1.4.0
Init docstring: __init__(self, predictionCol="prediction", labelCol="label",                  metricName="rmse")
File:           

In [22]:
evaluator = RegressionEvaluator(
    predictionCol='prediction',
    labelCol='rating',
    metricName='mae'
)

In [23]:
tvs = TrainValidationSplit(
    estimator=als,
    estimatorParamMaps=grid_search,
    evaluator=evaluator,
    trainRatio=0.8
)

In [24]:
tvsmodel = tvs.fit(train_df)

In [25]:
def get_best_param(model):
    result = [
        (
            [
                {key.name: param_value} for key, param_value in zip(
                    param.keys(), param.values())
            ], metric
        ) for param, metric in zip(
            model.getEstimatorParamMaps(),
            model.validationMetrics)  # validationMetrics  avgMetrics
    ]
    # ([{'maxIter': 50}, {'regParam': 0.01}], 0.7385557487596289)
    best_param = sorted(result, key=lambda e: e[1], reverse=True)[0]
    return best_param[0]

In [26]:
best_param = get_best_param(tvsmodel)

In [27]:
print(best_param)

[{'rank': 10}, {'maxIter': 5}, {'regParam': 0.1}, {'alpha': 0.5}]


In [28]:
best_model = tvsmodel.bestModel

In [31]:
rec = best_model.recommendForUserSubset(test_df, 1)

In [32]:
rec.show(10)

+----+-------------------+
|user|    recommendations|
+----+-------------------+
| 148|[[1159, 5.0788465]]|
| 463|   [[61, 4.082278]]|
| 471| [[1427, 5.779977]]|
| 496|[[1467, 4.2834516]]|
| 833| [[865, 4.5775824]]|
| 243|[[1398, 4.5864806]]|
| 392| [[1643, 5.123408]]|
| 540| [[169, 4.7467027]]|
| 623|[[1643, 5.2861347]]|
| 737|[[1193, 5.0169325]]|
+----+-------------------+
only showing top 10 rows

